<a href="https://colab.research.google.com/github/srilekhaj/Langchain-with-AI-agents/blob/main/cltv_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("cltv")

cltv


In [2]:
#preapre the data with customer grouping
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv("/content/sample_data/Ecommerce_Data.csv")
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,439570,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12
1,387281,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10
2,337863,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9
3,57628,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13
4,330897,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10


In [3]:
#some small data wrangling task

df['total_revenue'] = df['Quantity']*df['UnitPrice']
df.drop('Unnamed: 0', axis=1, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df[df['Quantity']>0]
df.head()


,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,total_revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9,45.00
3,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13,16.50
4,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10,90.00


In [4]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Find the first purchase date for each customer
first_purchase_date = df.groupby('CustomerID')['Date'].min()

# Find the last purchase date for each customer
last_purchase_date = df.groupby('CustomerID')['Date'].max()

# Calculate customer lifespan (in days)
customer_lifespan_days = (last_purchase_date - first_purchase_date).dt.days
customer_lifespan_days

,Date
CustomerID,
12347.0,365
12348.0,283
12349.0,0
12350.0,0
12352.0,260
...,...
18280.0,0
18281.0,0
18282.0,119


In [5]:
#grouping our data customer id and aggreegate the meterics
max_date = df['Date'].max()
data = df.groupby('CustomerID').agg(
    {
        'Date':lambda x: (max_date-x.min()).days,
        'InvoiceNo': lambda x: len(x),
        'Quantity': lambda x: x.sum(),
        'total_revenue': lambda x: x.sum()
    }
)

data

,Date,InvoiceNo,Quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33
...,...,...,...,...
18280.0,277,3,9,52.75
18281.0,180,2,11,33.45
18282.0,126,4,11,67.85


In [6]:
max_date

Timestamp('2011-12-09 00:00:00')

In [7]:
#change the name of the columns and insure that we dont have 0 quantity
data.columns= ['CustomerLifespanDays','num_transactions', 'quantity', 'total_revenue']
data = data[data['quantity']>0]

data

,CustomerLifespanDays,num_transactions,quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33
...,...,...,...,...
18280.0,277,3,9,52.75
18281.0,180,2,11,33.45
18282.0,126,4,11,67.85


In [8]:
#getting the elements of CLTV calculation

data['APV']  = data['total_revenue'] / data['num_transactions']
data


,CustomerLifespanDays,num_transactions,quantity,total_revenue,APV
CustomerID,,,,,
12347.0,367,48,623,1146.59,23.887292
12348.0,358,7,531,550.92,78.702857
12349.0,18,16,203,406.32,25.395000
12350.0,310,2,36,45.60,22.800000
12352.0,296,21,163,663.33,31.587143
...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333
18281.0,180,2,11,33.45,16.725000
18282.0,126,4,11,67.85,16.962500


In [9]:
#lets calculate the purchase frequency
purchase_freq = sum(data['num_transactions'])/len(data)
purchase_freq

19.496336101612115

In [10]:
repeat_rate  = data[data['num_transactions']>1].shape[0]/data.shape[0]
repeat_rate

0.9291646311675623

In [11]:
churn_rate = 1-repeat_rate
churn_rate


0.07083536883243768

In [12]:
data['profit_margin']  = data['total_revenue']* .10
data.head()

,CustomerLifespanDays,num_transactions,quantity,total_revenue,APV,profit_margin
CustomerID,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659
12348.0,358,7,531,550.92,78.702857,55.092
12349.0,18,16,203,406.32,25.395000,40.632
12350.0,310,2,36,45.60,22.800000,4.560
12352.0,296,21,163,663.33,31.587143,66.333


In [13]:
data['CLTV'] = ((data['APV']*purchase_freq)/churn_rate) *.10

data

,CustomerLifespanDays,num_transactions,quantity,total_revenue,APV,profit_margin,CLTV
CustomerID,,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659,657.460637
12348.0,358,7,531,550.92,78.702857,55.092,2166.174018
12349.0,18,16,203,406.32,25.395000,40.632,698.957969
12350.0,310,2,36,45.60,22.800000,4.560,627.534621
12352.0,296,21,163,663.33,31.587143,66.333,869.387093
...,...,...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333,5.275,483.953966
18281.0,180,2,11,33.45,16.725000,3.345,460.329672
18282.0,126,4,11,67.85,16.962500,6.785,466.866491


In [14]:
#some summary
import seaborn
print("The median CLTV is", data['CLTV'].median())
print("The average CLTV is", data['CLTV'].mean())

The median CLTV is 460.5410131773401
The average CLTV is 747.7434697480141


In [15]:
data.sort_values(by='CLTV').reset_index()
# Optionally, save the CLTV data to a CSV file
data.to_csv('customer_cltv_final.csv', index=False)

In [16]:
# segment this to idenity customer find important to grp them and segnemtn them



In [17]:
# Assuming you have already calculated CLTV and stored it in the 'data' DataFrame

# 1. Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Assuming you calculated 'profit_margin' in a previous step and it's available in your DataFrame.
# If not, you need to calculate or define it before using it in the features.
X = data[['CustomerLifespanDays', 'num_transactions', 'quantity', 'total_revenue', 'APV']]  # Features - Removed 'profit_margin'
# or
# X = data[['CustomerLifespanDays', 'num_transactions', 'quantity', 'total_revenue', 'APV','profit_margin']] # Features - Assuming 'profit_margin' is calculated and added to data
y = data['CLTV']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 2. Choose a regression model (e.g., Linear Regression, Random Forest Regressor)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Example using Random Forest Regressor
model = RandomForestRegressor()

# 3. Train the model on the training data
model.fit(X_train, y_train)

# 4. Make predictions on the testing data
y_pred = model.predict(X_test)

# 5. Evaluate the model's performance using metrics like R-squared, Mean Absolute Error (MAE), Root Mean Squared Error (RMSE)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"R-squared: {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

# You can further analyze the model's performance by:
# - Examining feature importance to understand which factors contribute most to CLTV prediction.
# - Plotting predicted vs. actual CLTV values to visualize the model's accuracy.
# - Performing cross-validation to obtain a more robust estimate of the model's performance.

# Example of feature importance using Random Forest
# feature_importances = model.feature_importances_
# print(feature_importances)



R-squared: 0.9947476102956043
Mean Absolute Error (MAE): 8.242630044812106
Root Mean Squared Error (RMSE): 107.52302820866012


In [18]:


# ... (rest of the code remains the same)

In [21]:
# prompt: give sample input to predict cltv

# Sample input data for predicting CLTV
# You can replace these values with your own data

sample_input = {
    'CustomerLifespanDays': [365, 180, 90, 367],
    'num_transactions': [5, 2, 1, 48],
    'quantity': [100, 50, 20,623],
    'total_revenue': [500, 250, 100,1146],
    'APV': [100, 125, 100,23.88]
}


# Create a DataFrame from the sample input
sample_df = pd.DataFrame(sample_input)

# Make predictions using the trained model
cltv_predictions = model.predict(sample_df)

# Print the predicted CLTV values
print(cltv_predictions)

[2736.97412853 3458.54382359 2728.88166898  657.23789423]
